In [3]:
import pandas
import pickle
import datetime
import time
import pytz
import numpy as np
import mplfinance as mpf
import matplotlib.ticker as mticker
import io
from PIL import Image
from pathlib import Path


In [4]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)


In [5]:
# GET AND DOWN_SELECT TRADES
x = pandas.read_pickle("../output/trades_with_derivatives.pickle")

distance = 0.50
rvol = 1.0
parameter_buffer = 0.1
parameter_stop = 0.3
parameter_target = 0.3
direction = 'long'

trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] < rvol) & (x['parameter_target'] == parameter_target) & (x['parameter_buffer'] == parameter_buffer) & (x['parameter_stop'] == parameter_stop) & (x['distance'] >= distance)]


KeyError: 'parameter_target'

In [ ]:
trades.head()

In [ ]:
pos = trades[trades['r'] > 0]
neg = trades[trades['r'] < 0]


In [ ]:
pos.shape

In [ ]:
neg.shape

In [ ]:
pos['tr_SMA_20min'].mean()

In [ ]:
neg['tr_SMA_20min'].mean()

In [ ]:
z = np.arange(0, 1.5, 0.05)
z = [round(q, 2) for q in z]
# x = [*zip(x,x[1:])]

pos.plot.hist(column = 'tr_SMA_20min', bins=z)

In [ ]:
z = np.arange(0, 1.5, 0.05)
z = [round(q, 2) for q in z]
# x = [*zip(x,x[1:])]

neg.plot.hist(column = 'tr_SMA_20min', bins=z)

In [ ]:
# GET AND DOWN_SELECT TRADES
x = pandas.read_pickle("../output/trades_with_derivatives.pickle")


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


# Trade count distribution over atr buckets

In [ ]:
st = 0.5
distance = 0.25
rvol = 1.0
parameter_buffer = 0.1
parameter_stop = st
parameter_target = st
direction = 'long'
trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] <= rvol) & (x['parameter_target'] == parameter_target) & (x['parameter_buffer'] == parameter_buffer) & (x['parameter_stop'] == parameter_stop) & (x['distance'] >= distance)]

pos = trades[trades['r'] > 0]
neg = trades[trades['r'] < 0]

z = np.arange(0, 1.75, 0.1)
z = [round(q, 2) for q in z]

# pos_vc = pos['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]
# neg_vc = neg['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]

bins = pandas.cut(pos['tr_SMA_20min'], z)
gb = pos.groupby(bins)['max_distance'].mean().iloc[::-1]

p = gb.plot(kind = 'barh')
plt.show()

# Compute and populate MAE

In [ ]:
def calc_mae(df):
    df = df[df['r'] > 0]
    
    if df.shape[0] == 0:
        return float('nan')
    
    return df['parameter_stop'].min()

def calc_all_mae(trades):
    columns = ['entry_time', 'entry_price', 'target_price', 'parameter_buffer']
    mae_groups = trades.groupby(columns).apply(calc_mae).to_frame('mae').reset_index()
    
    return trades.merge(mae_groups, how='left', left_on=columns, right_on=columns)

In [ ]:
trades = x
trades_with_mae = calc_all_mae(trades)

In [ ]:
trades_with_mae.head()

# ATR -> MAE

In [ ]:
# ATR -> MAE
for st in np.arange(0.1, 2.05, 0.1):
    st = round(st, 2)
    distance = 0.50
    rvol = 1.0
    parameter_buffer = 0.1
    parameter_stop = st
    parameter_target = st
    direction = 'long'
    trades = trades_with_mae[(trades_with_mae['direction'] == direction) & (trades_with_mae['rvol_5_SMA_15min'] <= rvol) & (trades_with_mae['parameter_target'] == parameter_target) & (trades_with_mae['parameter_buffer'] == parameter_buffer) & (trades_with_mae['parameter_stop'] == parameter_stop) & (trades_with_mae['distance'] >= distance)]

    pos = trades[trades['r'] > 0]
    neg = trades[trades['r'] < 0]

    z = np.arange(0, 1.75, 0.2)
    z = [round(q, 2) for q in z]

    # pos_vc = pos['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]
    # neg_vc = neg['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]

    bins = pandas.cut(pos['tr_SMA_20min'], z)
    gb = pos.groupby(bins)['mae'].median().iloc[::-1]
    
    print(f'target: {st:0.2f}')
    
    p = gb.plot(kind = 'barh')
    plt.show()


# ATR -> MFE

In [ ]:
for st in np.arange(0.1, 1.05, 0.1):
    st = round(st, 2)
    distance = 0.50
    rvol = 1.0
    parameter_buffer = 0.1
    parameter_stop = st
    parameter_target = st
    direction = 'long'
    trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] <= rvol) & (x['parameter_target'] == parameter_target) & (x['parameter_buffer'] == parameter_buffer) & (x['parameter_stop'] == parameter_stop) & (x['distance'] >= distance)]

    pos = trades[trades['r'] > 0]
    neg = trades[trades['r'] < 0]

    z = np.arange(0, 1.75, 0.2)
    z = [round(q, 2) for q in z]

    # pos_vc = pos['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]
    # neg_vc = neg['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]

    bins = pandas.cut(pos['tr_SMA_20min'], z)
    gb = pos.groupby(bins)['max_distance'].median().iloc[::-1]
    
    print(f'stop: {st:0.2f}')
    
    p = gb.plot(kind = 'barh')
    plt.show()


# Trade count over ATR

In [ ]:
distance = 0.25
rvol = 4.0
parameter_buffer = 0.1
parameter_stop = 0.5
parameter_target = 0.5
direction = 'long'
trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] <= rvol) & (x['parameter_target'] == parameter_target) & (x['parameter_buffer'] == parameter_buffer) & (x['parameter_stop'] == parameter_stop) & (x['distance'] >= distance)]

z = np.arange(0, 1.5, 0.1)
z = [round(q, 2) for q in z]

trades_vc = trades['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]

trades_vc.plot(kind = 'barh')
plt.show()
print()


# ATR -> Winrate (with constant target)

In [ ]:
for st in np.arange(0.1, 1.05, 0.1):
    st = round(st, 2)
    
    distance = 0.25
    rvol = 1.0
    parameter_buffer = 0.1
    parameter_stop = st
    parameter_target = 0.5
    direction = 'long'
    trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] <= rvol) & (x['parameter_target'] == parameter_target) & (x['parameter_buffer'] == parameter_buffer) & (x['parameter_stop'] == parameter_stop) & (x['distance'] >= distance)]

    pos = trades[trades['r'] > 0]
    neg = trades[trades['r'] < 0]

    z = np.arange(0.1, 0.65, 0.1)
    z = [round(q, 2) for q in z]

    pos_vc = pos['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]
    neg_vc = neg['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]

    ratio = len(pos) / len(neg) if len(neg) > 0 else 1
    delta = (pos_vc / (neg_vc + pos_vc))
    winrate = (pos.shape[0] / (neg.shape[0] + pos.shape[0]))
    print(f'stop: {st:0.2f} -> winrate: {winrate:0.2%} ttl: {trades.shape[0]:3} pos_neg_ttl_ratio:{pos.shape[0]:3}/{neg.shape[0]}')
    p = delta.plot(kind = 'barh')
    p.axvline(0.5, color="red", linestyle="--")
    plt.show()
    print()


# ATR -> Winrate

In [ ]:
for st in np.arange(0.1, 1.05, 0.1):
    st = round(st, 2)
    
    distance = 0.50
    rvol = 1.0
    parameter_buffer = 0.1
    parameter_stop = st
    parameter_target = st
    direction = 'long'
    trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] <= rvol) & (x['parameter_target'] == parameter_target) & (x['parameter_buffer'] == parameter_buffer) & (x['parameter_stop'] == parameter_stop) & (x['distance'] >= distance)]

    pos = trades[trades['r'] > 0]
    neg = trades[trades['r'] < 0]

    z = np.arange(0, 1.75, 0.2)
    z = [round(q, 2) for q in z]

    pos_vc = pos['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]
    neg_vc = neg['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]

    ratio = len(pos) / len(neg) if len(neg) > 0 else 1
    delta = (pos_vc / (neg_vc + pos_vc))
    winrate = (pos.shape[0] / (neg.shape[0] + pos.shape[0]))
    print(f'stop & target: {st:0.2f} -> winrate: {winrate:0.2%} ttl: {trades.shape[0]:3} pos_neg_ttl_ratio:{pos.shape[0]:3}/{neg.shape[0]}')
    p = delta.plot(kind = 'barh')
    p.axvline(0.5, color="red", linestyle="--")
    plt.show()
    print()


# Get most optimal trade from every combination of stop and target

In [ ]:
def calc_mae(df):
    df = df[df['r'] > 0]
    
    if df.shape[0] == 0:
        return float('nan')
    
    return df['parameter_stop'].min()

def calc_all_mae(trades):
    columns = ['entry_time', 'entry_price', 'target_price', 'parameter_buffer']
    mae_groups = trades.groupby(columns).apply(calc_mae).to_frame('mae').reset_index()
    
    return trades.merge(mae_groups, how='left', left_on=columns, right_on=columns)


# PARAMS
distance = 0.25
rvol = 1.0
parameter_buffer = 0.1
direction = 'long'
# END PARAMS

trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] <= rvol) & (x['parameter_buffer'] == parameter_buffer) & (x['distance'] >= distance)]
trades = trades[~trades['time_close']]
trades = trades[trades['r'] > 0]
trades = trades[trades['r'] > 0.3]
trades = trades[trades['parameter_stop'] > 0.3]
# trades = trades[trades['r'] <= 5]

columns = ['entry_time', 'entry_price', 'parameter_buffer']
best = trades.loc[trades.groupby(columns)['r'].idxmax()]


In [ ]:
best[best['tr_SMA_20min'] < 0.8].plot.scatter(x='tr_SMA_20min', y = 'parameter_stop')

In [ ]:
best[best['tr_SMA_20min'] < 0.8].plot.scatter(x='tr_SMA_20min', y = 'parameter_target')

In [ ]:
best[best['tr_SMA_20min'] < 0.8].plot.scatter(x='tr_SMA_20min', y = 'target_r')

In [ ]:
print(best['tr_SMA_20min'].corr(best['parameter_target']))
print(best['tr_SMA_20min'].corr(best['parameter_stop']))
print(best['tr_SMA_20min'].corr(best['target_r']))

In [ ]:
print(best['rvol_5_SMA_15min'].corr(best['tr_SMA_20min']))

In [ ]:
print(best['rvol_5_SMA_15min'].corr(best['parameter_target']))
print(best['rvol_5_SMA_15min'].corr(best['parameter_stop']))
print(best['rvol_5_SMA_15min'].corr(best['target_r']))

In [ ]:
print((best['tr_SMA_20min'] / best['rvol_5_SMA_15min']).corr(best['parameter_target']))
print((best['tr_SMA_20min'] / best['rvol_5_SMA_15min']).corr(best['parameter_stop']))
print((best['tr_SMA_20min'] / best['rvol_5_SMA_15min']).corr(best['target_r']))

In [ ]:
best['atr/rvol'] = best['tr_SMA_20min'] / best['rvol_5_SMA_15min']
best[best['atr/rvol'] < 1].plot.scatter(x='atr/rvol', y = 'parameter_target')

# Test correlations with outlier filtering

In [ ]:
def reduce(df, column, quantile):
    q_low = best[column].quantile(quantile)
    q_hi  = best[column].quantile(1 - quantile)

    best_reduced = best[(best[column] < q_hi) & (best[column] > q_low)]

    reduction_rate = 1 - (best_reduced.shape[0] / best.shape[0])
    print(f'{reduction_rate:0.2%} reduction')
    return best_reduced

In [ ]:
best_reduced = reduce(best, 'parameter_target', quantile = 0.01)
# best_reduced = reduce(best_reduced, 'parameter_stop', quantile = 0.01)
# best_reduced = reduce(best_reduced, 'target_r', quantile = 0.01)
# best_reduced = reduce(best_reduced, 'tr_SMA_20min', quantile = 0.01)

In [ ]:
best_reduced[best_reduced['tr_SMA_20min'] < 0.8].plot.scatter(x='tr_SMA_20min', y = 'parameter_stop')

In [ ]:
best_reduced[best_reduced['tr_SMA_20min'] < 0.8].plot.scatter(x='tr_SMA_20min', y = 'parameter_target')

In [ ]:
best_reduced[best_reduced['tr_SMA_20min'] < 0.8].plot.scatter(x='tr_SMA_20min', y = 'target_r')

In [ ]:
print(best_reduced['tr_SMA_20min'].corr(best_reduced['parameter_target']))
print(best_reduced['tr_SMA_20min'].corr(best_reduced['parameter_stop']))
print(best_reduced['tr_SMA_20min'].corr(best_reduced['target_r']))

In [ ]:
print(best_reduced['rvol_5_SMA_15min'].corr(best_reduced['tr_SMA_20min']))

In [ ]:
print(best_reduced['rvol_5_SMA_15min'].corr(best_reduced['parameter_target']))
print(best_reduced['rvol_5_SMA_15min'].corr(best_reduced['parameter_stop']))
print(best_reduced['rvol_5_SMA_15min'].corr(best_reduced['target_r']))

In [ ]:
print((best_reduced['tr_SMA_20min'] / best_reduced['rvol_5_SMA_15min']).corr(best_reduced['parameter_target']))
print((best_reduced['tr_SMA_20min'] / best_reduced['rvol_5_SMA_15min']).corr(best_reduced['parameter_stop']))
print((best_reduced['tr_SMA_20min'] / best_reduced['rvol_5_SMA_15min']).corr(best_reduced['target_r']))

In [ ]:
best_reduced['atr/rvol'] = best_reduced['tr_SMA_20min'] / best_reduced['rvol_5_SMA_15min']
best_reduced[best_reduced['atr/rvol'] < 1].plot.scatter(x='atr/rvol', y = 'parameter_target')

# Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
x = best_reduced[['tr_SMA_20min']].values.reshape(-1,1)
y = best_reduced[['parameter_target']].values.reshape(-1,1)

regression = LinearRegression().fit(x, y)
target_prediction = regression.predict(x)

plt.scatter(x, y, color = 'red')
plt.plot(x, target_prediction, color = 'blue')
plt.title('parameter_target vs tr_SMA_20min')
plt.xlabel('tr_SMA_20min')
plt.ylabel('parameter_target')
plt.show()
print(f'{regression.coef_[0, 0]:0.2f}*atr + {regression.intercept_[0]:0.2f}')


In [ ]:
x = best_reduced[['tr_SMA_20min']].values.reshape(-1,1)
y = best_reduced[['parameter_stop']].values.reshape(-1,1)

regression = LinearRegression().fit(x, y)
target_prediction = regression.predict(x)

plt.scatter(x, y, color = 'red')
plt.plot(x, target_prediction, color = 'blue')
plt.title('parameter_stop vs tr_SMA_20min')
plt.xlabel('tr_SMA_20min')
plt.ylabel('parameter_stop')
plt.show()
print(f'{regression.coef_[0, 0]:0.2f}*atr + {regression.intercept_[0]:0.2f}')


In [ ]:
# TMP

In [ ]:
target_prediction

# volume_SMA_20min CORRELATIONS

In [ ]:
best['volume_SMA_20min_x'] = best['volume_SMA_20min'] / 241150

In [ ]:
best.head()

In [ ]:
best[best['tr_SMA_20min'] < 0.8].plot.scatter(x='volume_SMA_20min_x', y = 'parameter_stop')

In [ ]:
best[best['tr_SMA_20min'] < 0.8].plot.scatter(x='volume_SMA_20min_x', y = 'parameter_target')

In [ ]:
best[best['tr_SMA_20min'] < 0.8].plot.scatter(x='volume_SMA_20min_x', y = 'target_r')

In [ ]:
print(best['volume_SMA_20min_x'].corr(best['parameter_target']))
print(best['volume_SMA_20min_x'].corr(best['parameter_stop']))
print(best['volume_SMA_20min_x'].corr(best['target_r']))

In [ ]:
print(best['volume_SMA_20min_x'].corr(best['tr_SMA_20min']))

In [ ]:
best[best['tr_SMA_20min'] < 0.9].plot.scatter(x='volume_SMA_20min_x', y = 'tr_SMA_20min')

In [ ]:
print((best['tr_SMA_20min'] * best['volume_SMA_20min_x']).corr(best['parameter_target']))
print((best['tr_SMA_20min'] * best['volume_SMA_20min_x']).corr(best['parameter_stop']))
print((best['tr_SMA_20min'] * best['volume_SMA_20min_x']).corr(best['target_r']))

In [ ]:
best['atr/vol'] = best['tr_SMA_20min'] / best['volume_SMA_20min_x']
best[best['atr/vol'] < 1].plot.scatter(x='atr/vol', y = 'parameter_target')

# distance CORRELATIONS

In [ ]:
best[best['tr_SMA_20min'] < 0.8].plot.scatter(x='distance', y = 'parameter_stop')

In [ ]:
best[best['tr_SMA_20min'] < 0.8].plot.scatter(x='distance', y = 'parameter_target')

In [ ]:
best[best['tr_SMA_20min'] < 0.8].plot.scatter(x='distance', y = 'target_r')

In [ ]:
print(best['distance'].corr(best['parameter_target']))
print(best['distance'].corr(best['parameter_stop']))
print(best['distance'].corr(best['target_r']))

In [ ]:
print(best['distance'].corr(best['tr_SMA_20min']))

In [ ]:
best[(best['tr_SMA_20min'] < 0.9) & (best['distance'] < 3.5)].plot.scatter(x='distance', y = 'tr_SMA_20min')

In [ ]:
print((best['distance'] * best['volume_SMA_20min_x']).corr(best['parameter_target']))
print((best['distance'] * best['volume_SMA_20min_x']).corr(best['parameter_stop']))
print((best['distance'] * best['volume_SMA_20min_x']).corr(best['target_r']))

In [ ]:
best['distance/vol'] = best['distance'] / best['volume_SMA_20min_x']
best[best['distance/vol'] < 1].plot.scatter(x='distance/vol', y = 'parameter_target')

# OPTIMAL BUCKETED VISUALIZATION

In [ ]:
best.head()

In [ ]:
def calc_mae(df):
    df = df[df['r'] > 0]
    
    if df.shape[0] == 0:
        return float('nan')
    
    return df['parameter_stop'].min()

def calc_all_mae(trades):
    columns = ['entry_time', 'entry_price', 'target_price', 'parameter_buffer']
    mae_groups = trades.groupby(columns).apply(calc_mae).to_frame('mae').reset_index()
    
    return trades.merge(mae_groups, how='left', left_on=columns, right_on=columns)


# PARAMS
distance = 0.25
rvol = 1.0
parameter_buffer = 0.1
direction = 'long'
# END PARAMS

trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] <= rvol) & (x['parameter_buffer'] == parameter_buffer) & (x['distance'] >= distance)]
trades = trades[~trades['time_close']]
trades = trades[trades['r'] > 0.3]
trades = trades[trades['r'] <= 10]
trades = calc_all_mae(trades)

columns = ['entry_time', 'entry_price', 'parameter_buffer']
best = trades.loc[trades.groupby(columns)['r'].idxmax()]


z = np.arange(0, 1.75, 0.2)
z = [round(q, 2) for q in z]

# pos_vc = pos['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]
# neg_vc = neg['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]

bins = pandas.cut(best['tr_SMA_20min'], z)
gb = best.groupby(bins)['max_distance'].median().iloc[::-1]

p = gb.plot(kind = 'barh')
plt.show()


In [ ]:
# BACKUP
# for st in np.arange(0.1, 1.05, 0.1):
#     st = round(st, 2)
    
#     distance = 0.50
#     rvol = 1.0
#     parameter_buffer = 0.1
#     parameter_stop = st
#     parameter_target = st
#     direction = 'long'
#     trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] <= rvol) & (x['parameter_target'] == parameter_target) & (x['parameter_buffer'] == parameter_buffer) & (x['parameter_stop'] == parameter_stop) & (x['distance'] >= distance)]

#     pos = trades[trades['r'] > 0]
#     neg = trades[trades['r'] < 0]

#     z = np.arange(0, 1.5, 0.2)
#     z = [round(q, 2) for q in z]

#     pos_vc = pos['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]
#     neg_vc = neg['tr_SMA_20min'].value_counts(bins = z, sort=False).iloc[::-1]

#     ratio = len(pos) / len(neg) if len(neg) > 0 else 1
#     delta = pos_vc - (neg_vc * ratio)
#     delta.plot(kind = 'barh')
#     plt.show()
#     winrate = (pos.shape[0] / (neg.shape[0] + pos.shape[0]))
#     print(f'stop & target: {st:0.2f} -> winrate: {winrate:0.2%} ttl: {trades.shape[0]:3} pos_neg_ttl_ratio:{pos.shape[0]:3}/{neg.shape[0]}')


In [ ]:

distance = 0.50
rvol = 1.0
parameter_buffer = 0.1
parameter_stop = 0.3
parameter_target = 0.3
direction = 'long'

trades = x[(x['direction'] == direction) & (x['rvol_5_SMA_15min'] < rvol) & (x['parameter_target'] == parameter_target) & (x['parameter_buffer'] == parameter_buffer) & (x['parameter_stop'] == parameter_stop) & (x['distance'] >= distance)]

pos = trades[trades['r'] > 0]
neg = trades[trades['r'] < 0]

z = np.arange(0, 1.5, 0.1)
z = [round(q, 2) for q in z]

pos_vc = pos['tr_SMA_20min'].value_counts(bins = z, sort=False)
neg_vc = neg['tr_SMA_20min'].value_counts(bins = z, sort=False)

delta = pos_vc - (neg_vc * (len(pos) / len(neg)))
delta.plot(kind = 'barh')